# Bayesian Optimization based on GP

In [3]:
import numpy as np
import sklearn.gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.gaussian_process.kernels import RBF, Matern
import math
import time
import multiprocessing as mp

## Gaussian Process-based Bayesian Optimization implemented
- Starting with expected improvement acq. function
- Going to add UCB

In [2]:
# EI takes the measured x-values and the gaussian process object as well as the current evaluated loss
# Boolean for maximizatioN/minimization
def EI(X, gaussian_process, current_loss, n_params, find_min = True, **args):
    
    X_pred = X.reshape(-1, n_params)
    mu, std = gaussian_process.predict(X_pred, return_cov = True)
    
    if find_min:
        best_loss = np.min(current_loss)
    else:
        best_loss = np.max(current_loss)
    
    # Normalize based on the GP posterior and account for max/min condition
    sign_X = (-1) ** find_min
    with np.errstate(divide = 'ignore'):
        norm_X = sign_X * (mu - best_loss)/std
        ei = mu * sign_X * (mu - best_loss) * norm.cdf(norm_X) + std * norm.pdf(norm_X)  
        
        # to exclude points with no standard deviation (likely alredy been tested)
        ei[std == 0] = 0
        return (-1) * ei

    
# UCB takes the same input as EI for simplicity, even though it does not need all of them
def UCB(X, gaussian_process, current_loss, n_params, find_min = True, kappa = 4):
    
    X_pred = X.reshape(-1, n_params)
    mu, std = gaussian_process.predict(X_pred, return_cov = True)
    
    sign_X = (-1) ** find_min
    ucb = mu + std * sign_X * kappa
    
    return ucb
    

In [3]:
# UCB takes the same input as EI for simplicity, even though it does not need all of them
def LP(X, X_eval, mu_eval, std_eval, grads, y_np):
    
    L = grads
    M = np.min(y_np)
    n_under_eval = X_eval.shape[0]
    # X, X_eval must be row vectors. Are they? Shape (n_under_eval, 2) otherwise reshape
    
    dists = np.array([np.linalg.norm(X - x) for x in X_eval])
    Z = [1 / (std_eval[i] * np.sqrt(2)) * (L * dists[i] - M + mu_eval[i]) for i in range(n_under_eval)]
    penalties = [0.5 * math.erfc((-1) * Z[i]) for i in range(n_under_eval)]
    
    return np.prod(penalties)

def HLP(X, X_eval,  mu_eval, std_eval, grads, gamma = 1):
    
    L = 2
    M = np.min(y_np)
    n_under_eval = X_eval.shape[0]
    # X, X_eval must be row vectors. Are they? Shape (n_under_eval, 2) otherwise reshape
    
    dists = np.array([np.linalg.norm(X - x) for x in X_eval])
    penalties = [dists[i] / (mu_eval[i] - M + gamma * std_eval[i] / L) for i in range(n_under_eval)]
    penalties = np.minimum(penalties, 1).tolist()

    return np.prod(penalties)


In [4]:
# same arguments as before, and bounds to limit the optimization as well as n_restarts to allow multiple optimization attempts
def sample_next_point(acq_func, gaussian_process, current_loss, find_min = True, bounds = (0, 1), n_restarts = 10):
    
    X_best = None
    best_acq_val = 999
    n_params = bounds.shape[0]
    
    starting_points = np.random.uniform(bounds[:,0], bounds[:,1], size = (n_restarts, n_params))
    for point in starting_points:
        result = minimize(acq_func, point.reshape(1, -1), method = 'L-BFGS-B', bounds = bounds, args = 
                          (gaussian_process, current_loss, n_params, find_min))
        
        if result.fun < best_acq_val:
            X_best = result.x
            best_acq_val = result.fun
            
    return X_best


# n_iters of attempts to optimize the objective, function, within bounds using n_random points to begin, or X_init
# alpha = variance of error term over GP, epsilon - precision tol for float
# three modes - standard (sequential evaluations), syncronous (new batches are evaluated with simultaneous start),
# and asynchronous (new points are evaluated before batch is finished)
def bayesian_optimization(n_iters, function, bounds, mode = 'standard', batch_penalty = LP,
                          X_init = None, n_init = 10, gp_params = None, find_min = True, alpha = 1e-5, epsilon = 1e-7, acq_func = EI):
    
    
    
    X_tested = []
    y_tested = []
    n_params = bounds.shape[0]
    
    if X_init is None:
        X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(function(X))
    
    X_np = np.array(X_tested)
    y_np = np.array(y_tested)
    
    # creating the gaussian process
    if gp_params is not None:
        gaussian_process = gp.GaussianProcessRegressor(**args) 
    
    else:
        kernel = Matern()
        gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, alpha = alpha, n_restarts_optimizer = 10, normalize_y = True)
    
    # CHANGES START HERE
    
    
    for n in range(n_iters):

        X_np = np.array(X_tested)
        y_np = np.array(y_tested)
        
        gaussian_process.fit(X_tested, y_tested)
        
        #sampling of next point - can be done with random search (not implemented) or optimization of act_func
        X_next = sample_next_point(acq_func, gaussian_process, y_tested, find_min, bounds = bounds, n_restarts = 10)
        
        while np.any(np.abs(X_next - X_tested) <= epsilon):
            X_next = sample_next_point(acq_func, gaussian_process, y_tested, find_min, bounds = bounds, n_restarts = 10)
            print(X_next, 'has already been sampled.\nIteration', n)
        
        y_next = function(X_next)
        X_tested.append(X_next)
        y_tested.append(y_next)
        
    return X_tested, y_tested

## Attempting to write logic for parallell

In [ ]:
for n in range(n_iters):

        X_np = np.array(X_tested)
        y_np = np.array(y_tested)
        
        gaussian_process.fit(X_tested, y_tested)
        
        # START THE ASYNC IMPLEMENTATION
        # create 3 lists of the 4 concurrent processes containing X, mu and std for each
        
        #sampling of next point - can be done with random search (not implemented) or optimization of act_func
        X_under_eval = []
        X_next = sample_next_point(acq_func, batch_penalty, X_under_eval,
                                   gaussian_process, y_tested, find_min, bounds = bounds, n_restarts = 10)
        
        while np.any(np.abs(X_next - X_tested) <= epsilon):
            X_next = sample_next_point(acq_func, gaussian_process, y_tested, find_min, bounds = bounds, n_restarts = 10)
            print(X_next, 'has already been sampled.\nIteration', n)
        
        y_next = function(X_next)
        X_tested.append(X_next)
        y_tested.append(y_next)
        
    return X_tested, y_tested
        
        
        # same arguments as before, and bounds to limit the optimization as well as n_restarts to allow multiple optimization attempts
def sample_next_parallel(acq_func, gaussian_process, current_loss, find_min = True, bounds = (0, 1), n_restarts = 10
                     batch_penalty = LP, local_L = True, X_under_eval = None):
    
    X_best = None
    best_acq_val = 999
    n_params = bounds.shape[0]
    
    starting_points = np.random.uniform(bounds[:,0], bounds[:,1], size = (n_restarts, n_params))
    
    if find_min:
        M = np.min(current_loss)
    else:
        M = np.max(current_loss)
        
    L = compute_L(gaussian_process, n_params, local_L)
    # Here, changes have to be made - consider the local penalty - L computed inside LP and HLP or here? Probably here. Then M should also be
    for point in starting_points:
        result = minimize(acq_func, point.reshape(1, -1), method = 'L-BFGS-B', bounds = bounds, args = 
                          (gaussian_process, current_loss, n_params, find_min))
        
        if result.fun < best_acq_val:
            X_best = result.x
            best_acq_val = result.fun
            
    return X_best


def compute_L(gaussian_process, n_params, local_L ,n_samples = 101):
    
    # set the distances between the samples to enable numeric gradient computation
    dists = [(bounds[i,1] - bounds[i,0]) / (n_samples - 1) for i in range(n_params)]
    mesh_X = [np.linspace(bounds[i,0], bounds[i,1], n_samples) for i in range(n_params)]
    
    all_X = np.array(np.meshgrid(*[mesh_X[i] for i in range(n_params)])).reshape(n_params,-1).T
    mu = gaussian_process.predict(all_X).reshape(n_samples, n_samples)
    
    if local_X is None:
        np.abs(np.gradient(mu, *dists)).max()
    
    else:        
        


## Testing the implementation on Branin-Hoo

In [2]:

def branin(X):

    result = np.square(X[1] - (5.1/(4*np.square(math.pi)))*np.square(X[0]) + 
         (5/math.pi)*X[0] - 6) + 10*(1-(1./(8*math.pi)))*np.cos(X[0]) + 10
    
    result = float(result)
    noise = np.random.normal() * 0.
    
    time_sleep = np.random.randint(1, 3)
    #print ('result:', result, '\nobserved:', noise + result, '\nSleeping for', time_sleep, 'seconds.')
    #time.sleep(time_sleep)
    return result + noise

In [ ]:
X_tested, y_tested = bayesian_optimization(200, branin, bounds = np.array([[-5, 10], [0, 15]]), n_init = 16, acq_func = UCB)

best_value = np.min(y_tested)
best_iter = np.argmin(y_tested)
best_X = X_tested[best_iter]
print(best_X, best_value, best_iter)


![Figure 1-1](Capture.png "Figure 1-1")

## Attempting to implement parallell evaluations
- evaluating the function is what takes time (implement time.sleep)
- Implement sync/async mode

In [9]:
from multiprocessing import Pool
from time import sleep
from workers import worker

results = []
def callback(worker, time):
    global results
    results.append((worker, time))
if __name__=="__main__":
pool = Pool(4)
a = [pool.apply_async(worker, (i, 5), callback=callback) for i in range(12)]

pool.close()
pool.join()
print(results)
print('Done')



[]
Done


In [99]:
kernel = Matern()
gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, n_restarts_optimizer = 10, normalize_y = True)


bounds = np.array([[-5, 10], [0, 15]])

n_params = bounds.shape[0]
n_init = 100
X_init = None
X_tested = []
y_tested = []

if X_init is None:
    X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, n_params))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(branin(X))
    
    X_np = np.array(X_tested)
    y_np = np.array(y_tested)

gaussian_process.fit(X_np, y_np)


n_samples = 101
dists = [(bounds[i,1] - bounds[i,0]) / (n_samples - 1) for i in range(n_params)]
mesh_X = [np.linspace(bounds[i,0], bounds[i,1], n_samples) for i in range(n_params)]
all_X = np.array(np.meshgrid(*[mesh_X[i] for i in range(n_params)])).reshape(n_params,-1).T
mu = gaussian_process.predict(all_X).reshape(n_samples, n_samples)
    
np.abs(np.gradient(mu, *dists)).max()



68.0140276160403

[0.075, 0.075]